# Cell/particle Counting and scoring stained objects

This notebook demonstrates how to process plates associated to the paper ['Integration of biological data by kernels on graph nodes allows prediction of new genes involved in mitotic chromosome condensation.'](http://dx.doi.org/10.1091/mbc.E13-04-0221) using [CellProfiler](http://cellprofiler.org/) version 3.1.3.
We use the example pipeline [Cell/particle counting, and scoring the percentage of stained objects](http://cellprofiler.org/examples/#PercentPositive). This pipeline is for two channels images only.

### Import Packages

In [ ]:
# %tb

# Import Cell Profiler Dependencies
import cellprofiler
import cellprofiler.preferences as cpprefs
import cellprofiler.module as cpm
import cellprofiler.pipeline as cpp
cpprefs.set_headless()

# Inject Image module used to inject OMERO image planes into Cell Profiler Pipeline
from cellprofiler.modules.injectimage import InjectImage

# Import Numpy
import numpy as np

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

# Import Matplotlib
import matplotlib
import matplotlib.pyplot as plt

### Set Cell Output Directory

In [ ]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### Create connection to IDR (Public Database : Read-only!)

In [ ]:
from idr import connection; 
conn = connection()
print conn

### Fetch the Plate that contains the Images to be analysed

In [ ]:
plate_id = 422
plate = conn.getObject("Plate", plate_id)
print 'Plate Name: ', plate.getName()

In [ ]:
# Load pipeline and inspect modules
from os.path import expanduser
home = expanduser("~")
pipeline = cpp.Pipeline()
pipeline.load(home+"/notebooks/CellProfiler/pipelines/ExamplePercentPositive.cppipe")

# Remove first 4 modules: Images, Metadata, NamesAndTypes, Groups...
# (replaced by InjectImage module below)
for i in range(4):
    print 'Remove module: ', pipeline.modules()[0].module_name
    pipeline.remove_module(1)

print 'Pipeline modules:'
for module in pipeline.modules():
    print module.module_num, module.module_name

### Run Cell Profiler Pipeline on a Plate

In [ ]:
warnings.filterwarnings('ignore')

Nuclei = pandas.DataFrame()
files = list()

# create list from generator
wells = list(plate.listChildren())
well_count = len(wells)

for count, well in enumerate(wells):
    print 'Well: %s/%s' % (count + 1, well_count), 'row:', well.row, 'column:', well.column
    # Load a single Image per Well
    image = well.getImage(0)
    pixels = image.getPrimaryPixels()
    size_c = image.getSizeC()
    
    # For each Image in IDR, we copy the pipeline and inject image modules
    pipeline_copy = pipeline.copy()
    
    # Inject image for each Channel (pipeline only handles 2 channels)
    for c in range(0, size_c):
        
        plane = pixels.getPlane(0, c, 0)
        image_name = image.getName()
        
        # Name of the channel expected in the pipeline
        if c == 0:
            image_name = 'OrigBlue'
        if c == 1:
            image_name = 'OrigGreen'
        
        inject_image_module = InjectImage(image_name, plane)
        inject_image_module.set_module_num(1)
        pipeline_copy.add_module(inject_image_module)
    
    m = pipeline_copy.run()
    
    #Results obtained as CSV from Cell Profiler
    path = new_output_directory + '/Nuclei.csv'
    f = pandas.read_csv(path, index_col=None, header=0)
    f['Image'] = image.getId()
    f['Well'] = well.getId()
    f['Cell_Count'] = len(f.index)
    files.append(f)

Nuclei = pandas.concat(files, ignore_index=True)

### Calculate statistics

In [ ]:
Nuclei.describe()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (32.0, 30.0)
#Drop few columns
df = Nuclei.drop(['Image', 'ImageNumber', 'Well', 'ObjectNumber', 'Number_Object_Number', 'Classify_PH3Neg', 'Classify_PH3Pos'], axis=1)
df.hist();

<br>
### Close the connection

In [ ]:
conn.close()